In [ ]:
import numpy as np

import pandas as pd

import xarray as xr

import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

from math import e

import datetime

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from random import seed
from random import randint

from scipy.signal import hilbert
import statistics
from scipy.stats import stats
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
import pickle
import cartopy.crs as ccrs
import matplotlib.ticker as ticker
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from time import time
import scipy.stats as stats
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
fogData = xr.open_dataset('../../../Data/FogData/CombinedFogData_25Stations.nc')
#plt.figure(figsize=[16,8])


In [ ]:
StackFog=fogData.fogdata.stack(a=('years','months','days'))
StackFog
dd =[];
for i in range(StackFog.years.values.shape[0]):
    dd=dd+[str(StackFog.years[i].values)+'-'+str(StackFog.months[i].values)+"-"+str(StackFog.days[i].values)]
fg = xr.Dataset({'fogdata': (('time','stations'), StackFog.values.T)}, coords={'time': pd.to_datetime(dd),'stations': fogData.stations})
# fogData.fogdata.values.shape
# fogData.stations
#plt.figure(figsize=[16,8])
#fg.fogdata.resample(time="1y").sum().sum(dim='stations').plot()
#fg.fogdata.time    

In [ ]:
t2mData = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_t2m_3hourly_NovDecJan.nc')
u10Data = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_u10_3hourly_NovDecJan.nc')
v10Data = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_v10_3hourly_NovDecJan.nc')
d2mData = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_d2m_3hourly_NovDecJan.nc')
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
d2mD = d2mData.sel(latitude=slice(32,24),longitude=slice(74,86))

In [ ]:
# Calculate wind speed and relative humidity

ws = ((v10D.v10[:,0,:,:].values**2)+(u10D.u10[:,0,:,:].values**2))**0.5

ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh = (e**((17.625*(d2mD.d2m[:,0,:,:].values-273.15))/(243.04+(d2mD.d2m[:,0,:,:].values-273.15)))/e**((17.625*(t2mD.t2m[:,0,:,:].values-273.15))/(243.04+(t2mD.t2m[:,0,:,:].values-273.15))))

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rh)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#rh_ds


In [ ]:
# Read 850 data

tz850Data = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_850_tqz_3hourly_NovDecJan.nc')

uv850Data = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_850_uv_3hourly_NovDecJan.nc')

tz850D = tz850Data.sel(latitude=slice(32,24),longitude=slice(74,86))

uv850D = uv850Data.sel(latitude=slice(32,24),longitude=slice(74,86))

In [ ]:
#Calculate inv

inv=t2mD.t2m[:,0,:,:].values-tz850D.t[:,0,:,:].values

inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

inv_ds.attrs

inv_ds.attrs['units']='K'

inv_ds.attrs['long_name']='t2m - t850'

#inv_ds.inv

In [ ]:
#u shear calculation

ushear=(uv850D.u[:,0,:,:].values-u10D.u10[:,0,:,:].values)/(tz850D.z[:,0,:,:].values/9.81) 

ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

ushear_ds.attrs['units']='s-1'

ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [ ]:

w700Data = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/DailyERA5IGP_700_w_3hourly_NovDecJan.nc')

w700D = w700Data.sel(latitude=slice(32,24),longitude=slice(74,86))

# Read Orography

OroData = xr.open_dataset('../../../Data/IGPERA5/DailyFiles/ERA5IGP_Orography.nc')

OroData.data_vars

OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))

oro = OroD.z

oro.values = OroD.z.values/9.81

oro.attrs

oro.attrs['units']='meter'

oro.attrs['long_name']='Orography'

l = np.arange(0,1000,100)

#oro[0,:,:].plot(levels=l,extend='both',cmap='jet')

In [ ]:
# Create Mask altitude >500 m NaN

oro.values[oro.values>500.1]=np.NaN

mask=oro.values/oro.values

# chk mask

t2mD.t2m.values=t2mD.t2m.values*mask

#t2mData.t2m[0,0,:,:].plot(cmap='jet')

#plt.figure(figsize=[20,10])

#t2mData.mean(dim=['latitude','longitude']).t2m[:,0].plot()

t2mTS=t2mD.mean(dim=['latitude','longitude'], skipna=True).t2m[:,0]

In [ ]:
ws_ds.ws.values=ws_ds.ws.values*mask
wsTS=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
#wsTS.plot()
rh_ds.rh.values=rh_ds.rh.values*mask
#rh_ds.rh[:].mean(dim=['latitude','longitude'], skipna=True).plot()
rhTS=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
rhTS.plot()
w700D.w.values=w700D.w.values*mask
w700D.mean(dim=['latitude','longitude'])
wTS=w700D.mean(dim=['latitude','longitude'], skipna=True).w[:,0]
#wTS.plot()
inv_ds.inv.values=inv_ds.inv.values*mask
invTS=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
#invTS.plot()
ushear_ds.ushear.values=ushear_ds.ushear.values*mask
ushearTS=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear
#ushearTS.plot()
#Create area averaged values and get same days as fog days i.e. Dec Jan

# a,b = xr.align(fg.fogdata.sum(dim='stations'),t2mTS)

t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

#t2m

In [ ]:

X       = pd.DataFrame([t2m.values,ws.values,rh.values,inv.values,w.values,ushear.values*100.0]).T[:]
X.index = pd.to_datetime(t2m.time.values)
X

In [ ]:
y=fg.fogdata.sum(dim='stations').sel(time=slice('1980-1-1','2014-12-31'))
y=y/25.0
y

In [ ]:
# build a classifier
gbReg = GradientBoostingRegressor(random_state=0)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
#=20, max_depth= 3,learning_rate=0.04,
param_dist = {'n_estimators':[300,500,1000,1500,2000,2500,3000],
              'min_samples_leaf': [5,10,15,20],
              'max_depth': [2,3,4],
              'learning_rate': [0.1,0.2,0.3,0.05,0.03,0.02,0.01]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(gbReg, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X, y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)


In [ ]:
# build a classifier
gbReg = GradientBoostingRegressor(random_state=0)


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
#=20, max_depth= 3,learning_rate=0.04,
param_dist = {'n_estimators':[300,500,1000],
              'min_samples_leaf': [5,10,15],
              'max_depth': [2,3,4],
              'learning_rate': [0.1,0.3,0.05,0.01]}

# run grid search
grid_search = GridSearchCV(gbReg, param_grid=param_dist,scoring='r2',cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)


# SVR

In [ ]:
from sklearn.svm import SVR

In [ ]:
seed(0)
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.20, random_state=0)

In [ ]:
regressor = SVR(C= 5, gamma='auto', kernel= 'rbf')

regressor.fit(X_train, y_train)

In [ ]:
Y_prediction_rmm1 = regressor.predict(X_test)

acc_svr       = round(regressor.score(X_test, y_test) * 100, 2)

In [ ]:
print("Accuracy of base SVR model is ,",(acc_svr/100)**0.5)

In [ ]:
svReg = SVR()


# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
#=20, max_depth= 3,learning_rate=0.04,
param_dist = {'kernel':[ 'rbf' ],
              'gamma': ['scale', 'auto'],
              'C': [1,2,3,4,5,6],
              }

# run grid search
grid_search = GridSearchCV(svReg, param_grid=param_dist,scoring='r2',cv=5)
start = time()
grid_search.fit(X, y)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)

# GB 

In [ ]:
regressor=GradientBoostingRegressor(n_estimators=300, min_samples_leaf=20, max_depth= 3,learning_rate=0.04,random_state=0)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
Y_prediction_rmm1 = regressor.predict(X_test)

acc_svr       = round(regressor.score(X_test, y_test) * 100, 2)

In [ ]:
print("Accuracy of GB model is ,",(acc_svr/100)**0.5)

# XGB

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor

In [ ]:
# define the model
model = XGBRegressor(n_estimators=100,learning_rate=0.05,max_depth=5,objective ='reg:squarederror' )


In [ ]:
model.fit(X_train, y_train.values)

In [ ]:
acc_svr       = round(model.score(X_test, y_test) * 100, 2)
print("Accuracy of XGB model is ,",(acc_svr/100)**0.5)

In [ ]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [ ]:
#  CNN

In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten,MaxPooling2D,AveragePooling2D